In [25]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding,Bidirectional,GRU, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [26]:
data=pd.read_csv("../input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv")
data.clean_text=data.clean_text.astype(str)



data = data[['clean_text','category']]

In [27]:
data=data[data.category !=0]
data['clean_text']=data['clean_text'].apply(lambda x: x.lower())
data['clean_text'] = data['clean_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['clean_text'].values)
X = tokenizer.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X)

print(data[ data['category'] == 1].size)
print(data[ data['category'] == -1].size)

In [28]:
embed_dim = 128
gru_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(GRU(gru_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [29]:
Y = pd.get_dummies(data['category']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [30]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size, verbose = 2)

In [37]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [39]:
text = ['Aditya is a very good boy']
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(text)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")